# Forecasting Homeless Shelter Demand for Effective Resource Allocation

### Data Science 3000 Group Project

By: Aaron Serro, Kalan Roye, Jisa Pius, Tina Movahedi


### Step 1: Load the Data


In [4]:
import pandas as pd
df = pd.read_csv("homelessness_shelter_data.csv")
df.head()

,id,date,shelter_name,city,state,total_capacity,occupied_beds,available_beds,occupancy_rate,average_age,male_percentage,female_percentage,season,notes
0,1,2023-10-05,Sunrise Shelter,Dallas,TX,85,62,23,72.9,19,43,57,Spring,High demand
1,2,2023-08-01,Safe Haven,Dallas,TX,128,117,11,91.4,58,45,55,Summer,No issues reported
2,3,2025-04-27,Second Chance,San Jose,CA,190,144,46,75.8,56,52,48,Summer,Excellent condition
3,4,2024-10-05,Harbor Home,San Jose,CA,115,7,108,6.1,63,48,52,Autumn,Needs maintenance
4,5,2024-09-15,Harbor Home,San Antonio,TX,155,65,90,41.9,21,48,52,Summer,No issues reported
